# Entanglement entropy in ground states

$\require{physics}
\def\bm{\boldsymbol}$

## Introduction

The Shannon entropy $H$ of a random variable $p$ over states $\{ \alpha \}$ is:
\begin{align}
    H [p] = - \sum_{\alpha} p (\alpha) \log p (\alpha).
\end{align}
A natural generalization of this concept to a wavefunction $\ket{\psi}$ (also a
probability distribution) with a singular value decomposition:
\begin{align}
    \ket{\psi} = \sum_{\alpha} \lambda_\alpha \ket{u_\alpha} \otimes \ket{v_\alpha}
\end{align}
where $\ket{u_\alpha} \otimes \ket{v_\alpha}$ are Schmidt vectors and 
$\lambda_\alpha$ are Schmidt values, is to define the entanglement entropy $S$:
\begin{align}
    S[\psi] = - \sum_\alpha \lambda_\alpha^2 \log \lambda_\alpha^2.
\end{align}

## Program

### Choosing subsystems

We will perform a Schmidt decomposition of a vector onto a subsystem of the 
spin chain of the TFIM model we are exploring in this class.
I won't say anything about the theory or mathematics of this decomposition,
since it is already explained well in the assignment, however I will mention
that my code can compute the Schmidt decomposition onto ANY subsystem of the
chain.
This required some thought, because as Brenden hinted during office hours, the
matricization operation reduces to reshaping a vector into a matrix as long as
the subsystem being matricized is a contiguous stretch of the fastest-changing 
bits in the computational basis.

The only problem in handling the arbitrary case of any subsystem is to apply
the correct permutation to the state vector in order for the subsystem of
importance to be in the position of fastest-changing bits.
(The actual position of those bits depends on the array storage format,
which numpy defaults to row-major ordering (though it can switch to columns).)
A few permutations in cycle form later, voilá, a fully capable implementation
is in place.
All one needs to do is to supply a list of the bit positions as the argument
`A` to the `basis.schmidt.matricize(L, A, v)` function, or any other function
from that module.

### Tasks

- For various $L$ and representative values of $h$ at open boundary conditions
calculate the ground state entanglement entropy for the fastest-changing
$\ell$ bits, with $1 \leq \ell \leq L-1$.
- Summary plot of the entanglement entropy versus $L$ at $\ell = L/2$
- For the largest system size, fit $S(\ell, L)$ at $h=1$ to
\begin{align}
    S(\ell; L) =
        \frac{c}{3} \log \left( \frac{L}{\pi} \sin \frac{\pi\ell}{L} \right) + C
\end{align}
- Repeat the above for the highest excited state

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import models, tests, basis, measure, data

In [ ]:
%%time
entropies = {
    'L' : [],
    'h' : [],
    'l' : [],
    'S' : [],
    'k' : [],
    'bc': [],
}

for oper_params in tests.tfim_sweep(
    L = [8, 10, 12, 14, 16, 18, 20],
    h = [0.3, 1, 1.7],
    bc= ['o', 'c'],
):
    job = dict(
        oper=models.tfim_z.H_sparse,
        oper_params=oper_params,
        solver=sla.eigsh,
        solver_params={ 
            'k' : 6, 
            'which' : 'BE',
        },
    )
    evals, evecs = data.jobs.obtain(**job)
    gs = evecs[:, 0]
    es = evecs[:, job['solver_params']['k'] - 1]
    A = []
    for l in range(oper_params['L']-1):
        A.append(l)
        for k, state in zip([0, job['solver_params']['k'] - 1], [gs, es]):
            entropies['S'].append(
                measure.entropy.entanglement(
                    basis.schmidt.values(state, A, oper_params['L'])
                )
            )
            entropies['l'].append(l)
            entropies['L'].append(oper_params['L'])
            entropies['h'].append(oper_params['h'])
            entropies['k'].append(k)
            entropies['bc'].append(oper_params['bc'])
        
df = pd.DataFrame(entropies)

In [ ]:
def plot_script(df, bc, k):
    """Make the display plots."""
    L = sorted(set(df.L))
    h = sorted(set(df.h))
    w = 2

    fig_l, axes_l = plt.subplots(len(L)//w+len(L)%w, w)
    for i, row in enumerate(axes_l):
        for j, ax in enumerate(row):
            if w*i + j < len(L):
                for s in h:
                    sub = df[(df.h==s) & (df.L==L[w*i+j]) & (df.bc==bc) & (df.k==k)]
                    ax.plot(sub.l.values, sub.S.values, label='h='+str(s))
                ax.set_xlabel(f'$l$ at $L={L[w*i+j]}$')
                ax.set_ylabel('$S$')
                handles, labels = ax.get_legend_handles_labels()
            else:
                ax.set_axis_off()
                ax.legend(handles, labels, loc='center')
    st = fig_l.suptitle('Entanglement entropies')
    fig_l.set_size_inches(5, 6)
    fig_l.tight_layout()
    st.set_y(.95)
    fig_l.subplots_adjust(top=.9)

    fig_L, ax_L = plt.subplots()
    for s in h:
        sub = df[(df.h==s) & (df.l==df.L//2) & (df.bc==bc) & (df.k==k)]
        ax_L.plot(sub.L.values, sub.S.values, label='h='+str(s))
    ax_L.set_title('Entanglement entropy at half-subsystem')
    ax_L.set_xlabel('$L$')
    ax_L.set_ylabel('$S$')
    ax_L.legend()
    fig_L.set_size_inches(3, 3)

    return (fig_l, fig_L)

## Results

We are ready to show some of these calculated results.

### Values

In [ ]:
df.head()

### Open system, ground state

Next we will show the plots of the entanglement entropy

In [ ]:
%%capture
plots = plot_script(df, 'o', 0)

In [ ]:
plots[0]

The plot above shows the entanglement entropy as a function of system sizes,
subsystem sizes, and the ordering parameter.
In the small $h<1$ regime, the entanglement is largest and relatively constant
with respect to the system size.
In the large $h>1$ regime, the entropy is smallest, indicating states that
are more classical, resembling product states.
At the critical point, the entropy is maximized at the halfway subsystem and
minimized at the largest, resp smallest subsystems.


The next plot shows the summary over $L$.
We can see that in the ferromagnetic and paramagnetic regions, the entropy is
essentially constant with respect to the chain length.
However at the critical point the entropy is increasing with length, because
there are more ways to entangle more particles in a way that is possible at the
phase transition.

In [ ]:
plots[1]

### Periodic system, ground state

In [ ]:
%%capture
plots_c = plot_script(df, 'c', 0)

In [ ]:
plots_c[0]

It is quite notable that the periodic boundary condition leads to double the
entanglement entropy at the critical point compared to the open system.
In this case, it is much larger because the bonds can extend the same length 
in both directions without the open boundary.
In the ferromagnetic and paramagnetic phases, the entropy is relatively unchanged.

In [ ]:
plots_c[1]

#### Critical point entropy area law

Next we fit the data at $L=20$ to this function:
\begin{align}
    S(\ell; L) =
        \frac{c}{3} \log \left( \frac{L}{\pi} \sin \frac{\pi\ell}{L} \right) + C
\end{align}
using the natural logarithm (as does `measure.entropy.entanglement()`).

In [ ]:
sub = df[(df.h==1) & (df.L==20) & (df.bc=='c') & (df.k==0)]

m, b, r, p, err = linregress(
    np.log((20 / np.pi) * np.sin((np.pi / 20) * (sub.l.values + 1))) / 3,
    sub.S.values,
)
print('slope: ', m)
print('yintr: ', b)
print('corrl: ', r)
print('pvalu: ', p)
print('stder: ', err)

These appear to be very significant results with log error, suggesting 
goodness of fit.
It's worth mentioning the the slope and y intercept are about the same value and
are highly correlated, but that might not be physically relevant.

### Open system, most excited state

Let's look at the other end of the spectrum

In [ ]:
%%capture
plots_o = plot_script(df, 'o', 5)

In [ ]:
plots_o[0]

In [ ]:
plots_o[1]

As before the values for the entropies are quite similar to the other end of the
spectrum.
This might be the case because the highly excited states will have most spins
antialigned, which can be reflected in the singular value decomposition without
adding much more information about the state - so the state looks like a ground
state except with more and regularly spaced sign flips.
In this open system and excited state, the area law, which says the entanglement
entropy is constant as a function of size (in 1D with ends as boundaries), 
because this is just about as unique a state as the ground state (except for the
antialignment) which means that one expects not much to change about the 
information content and entropy of the state.

#### Critical point entropy area law

We'll do a similar fit for the highest energy state

In [ ]:
sub = df[(df.h==1) & (df.L==20) & (df.bc=='o') & (df.k==5)]

m, b, r, p, err = linregress(
    np.log((20 / np.pi) * np.sin((np.pi / 20) * (sub.l.values + 1))) / 3,
    sub.S.values,
)
print('slope: ', m)
print('yintr: ', b)
print('corrl: ', r)
print('pvalu: ', p)
print('stder: ', err)

Because the boundaries are open, the slope and intercept are about half of 
what they were before, but this is quite similar to the previous case, including
the correlation of the slope and intercept.

## Discussion

In summary, we have seen how the physics of subsystems gives rise to the 
phenomenon of entropy, and have created evidence to verify the area law scaling
of the entanglement entropy in the 1D TFIM system.